# Învățare Automată
# Arbori de decizie. Păduri aleatoare
### Autori:
* Tudor Berariu - 2016
* George Muraru - 2020

## 1. Scopul laboratorului

Scopul laboratorului îl reprezintă întelegerea conceptului de arbore de decizie și implementarea unor clasificatori bazați pe acest model.

## 2. Problema de rezolvat

Problema de rezolvat ı̂n acest laborator este una de ı̂nvățare supervizată: fiind dat un **set de date X** ce conține exemple descrise printr-un set de **atribute discrete A** și etichetate cu **câte o clasă dintr-o mulțime cunoscută C**, să se construiască un model pentru clasificarea exemplelor noi.

## 3. Arbore de decizie


Un arbore de decizie este un clasificator ce aproximează funcții discrete.

Într-un arbore de decizie există 2 tipuri de noduri:
* *noduri intermediare* - conține un test pentru un atribut și are câte un arc (și implicit un subarbore) pentru fiecare valoare posibiliă a atributului
* *noduri frunză* - este etichetat cu o clasă

Pentru **a clasifica un obiect nou** se pornește din rădăcina arborelui și din fiecare nod se coboară pe arcul corespunzător valorii atributului pe care o are obiectul dat. Atunci când se ajunge ı̂ntr-un nod frunză, clasa acestuia va reprezenta predicția arborelui.

## 4. Păduri de arbori aleatori

*Pădurile de arbori aleatori* (eng. Random Forest) este un model format din mai mulți arbori de decizie.

Se bazează pe 2 hiperparametrii:
* Eșantionare aleatoare din setul de date de antrenament
* Subseturi aleatoare de atribute considerate la împărțirea pe mai multi subarbori

Predicția, utilizând un astfel de model, se bazează pe clasa majoritară oferită de predicțiile indepente ale tuturor arborilor.

## 5. Workspace Setup

### Câteva biblioteci de care vom avea nevoie

In [2]:
from math import log2
import csv
import os

### Hiperparametrii necesari rulării

In [3]:
DATASET_NAME = 'Tennis'  #@param ['Chess', 'Car', 'Tennis']

# Adâncimea arborilor
D = 3 #@param {type: "slider", min: 2, max: 10}

# Procentul de exemple din setul de date utilizat la construcția arborilor
P = 50 #@param {type: "slider", min: 1, max: 100}

### Funcții ajutătoare pentru descărcarea și lucrul cu setul de date

In [4]:
class Node:
    """ Representation for a node from the decision tree """
    def __init__(self, label):
        """
            for non-leafs it is the name of the attribute
            for leafs it is the class
        """
        self.label = label
        
        # Dictionary of (attribute value, nodes)
        self.children = {}
    
    def display(self, string):
        print(string + self.label)
        string += "\t"
        if self.children:
            for key, value in self.children.items():
                print(string + key)
                value.display(string + "\t")


def getArchive(dataSetName):
    """ Checks if a specific dataset is present in the local directory, if not,
    downloads it.

    Args:
        dataSetName (str): the dataset name
    """
    dataset_file = "datasets" + os.sep + dataSetName.lower()
    print(dataset_file)
    
    from os import path
    if not path.isfile(dataset_file):
        import urllib
        print("Downloading...")
        urllib.request.urlretrieve(dataset_url, filename=dataset_file)
        assert(path.isfile(dataset_file))
        print("Got the archive")
    else:
        print(f"{dataset_file} already in the local directory")


def getDataSet(dataSetName):
    """ Reads a dataset

    Args:
        dataSetName (str): Name for the dataset

    Returns:
        A tuple containing (classes, attributes, examples):
        classes (set): the classes that are found in the dataset
        attributes (list of strings): the attributes for the dataset
        examples (list of dictionaries): one example contains an entry as
            (attribute name, attribute value)
    """

    dataset_file = "datasets" + os.sep + dataSetName.lower()

    f_in = open(dataset_file, 'r')
    csv_reader = csv.reader(f_in, delimiter=",")

    # Read the header row
    row = next(csv_reader)

    # The last element represents the class
    attributeNames = row[:-1]
    
    examples = []
    classes = set()

    for row in csv_reader:
        *attributes, label = row
        classes.add(label)
        example = dict(zip(attributeNames, attributes))
        example["CLASS"] = label
        examples.append(example)
    
    f_in.close()
    return classes, attributeNames, examples

### Descărcare și încarcare set de date

In [5]:
getArchive(DATASET_NAME)
classes, attributes, examples = getDataSet(DATASET_NAME)
examples
attributes

datasets/tennis
datasets/tennis already in the local directory


['Outlook', 'Temperature', 'Humidity', 'Windy']

## 6. Cerințe

1. [3 pct] Implementați o funcție recursivă *randomTree* care construiește arbori de decizie de adâncime **d** pe baza unui **set de date X** și a unei **mulțimi de atribute A** astfel:
 * Dac̆a *d = 0*, atunci se construiește un nod frunză cu clasa majoritară din X.
 * Dacă *d > 0*, atunci se alege aleator un atribut $a_i$ din A și se construiește câte un subarbore pentru fiecare valoare $v_j$ a atributului $a_i$ apelând *randomTree* pentru *d − 1*:
$$
X_{i/j} = \{x \in X|a_{i}(x) = v_k\}\\
A_{new} = A \setminus \{a_i\}
$$

In [6]:
print(attributes)
print(classes)
print(examples)

['Outlook', 'Temperature', 'Humidity', 'Windy']
{'no', 'yes'}
[{'Outlook': 'sunny', 'Temperature': 'hot', 'Humidity': 'high', 'Windy': 'false', 'CLASS': 'no'}, {'Outlook': 'sunny', 'Temperature': 'hot', 'Humidity': 'high', 'Windy': 'true', 'CLASS': 'no'}, {'Outlook': 'overcast', 'Temperature': 'hot', 'Humidity': 'high', 'Windy': 'false', 'CLASS': 'yes'}, {'Outlook': 'rain', 'Temperature': 'mild', 'Humidity': 'high', 'Windy': 'false', 'CLASS': 'yes'}, {'Outlook': 'rain', 'Temperature': 'cool', 'Humidity': 'normal', 'Windy': 'false', 'CLASS': 'yes'}, {'Outlook': 'rain', 'Temperature': 'cool', 'Humidity': 'normal', 'Windy': 'true', 'CLASS': 'no'}, {'Outlook': 'overcast', 'Temperature': 'cool', 'Humidity': 'normal', 'Windy': 'true', 'CLASS': 'yes'}, {'Outlook': 'sunny', 'Temperature': 'mild', 'Humidity': 'high', 'Windy': 'false', 'CLASS': 'no'}, {'Outlook': 'sunny', 'Temperature': 'cool', 'Humidity': 'normal', 'Windy': 'false', 'CLASS': 'yes'}, {'Outlook': 'rain', 'Temperature': 'mild', 'H

In [22]:
import random
import statistics
from copy import deepcopy

getArchive(DATASET_NAME)
classes, attributes, examples = getDataSet(DATASET_NAME)

def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency > counter): 
            counter = curr_frequency 
            num = i 
  
    return num 

def randomTree(d, X, A):
   # Cerință 1
    if d == 0:
       mostFrequentClass = most_frequent(list(x["CLASS"] for x in X))
       n = Node(mostFrequentClass)
    else:
        attribute = random.choice(A)
        n = Node(attribute)
        A.remove(attribute)

        sett = []
        for i in range(len(X)):
            sett.append(X[i][attribute])
        atr_values = set(sett)

        for val in atr_values:
          if d == 1 : 
            X_prim = []
            for i in range(len(X)):
              if val in list(X[i].values()):
                X_prim.append(X[i])
            n.children[val] = randomTree(d - 1, X_prim, deepcopy(A))
          else :
            n.children[val] = randomTree(d - 1, X, deepcopy(A))

    return n

n = randomTree(3, examples, attributes)
print(n.display(""))

datasets/tennis
datasets/tennis already in the local directory
Windy
	false
		Temperature
			hot
				Outlook
					rain
						yes
					overcast
						yes
					sunny
						no
			mild
				Outlook
					rain
						yes
					overcast
						yes
					sunny
						no
			cool
				Humidity
					high
						no
					normal
						yes
	true
		Outlook
			rain
				Temperature
					hot
						no
					mild
						yes
					cool
						yes
			overcast
				Temperature
					hot
						no
					mild
						yes
					cool
						yes
			sunny
				Temperature
					hot
						no
					mild
						yes
					cool
						yes
None


2. [3 pct] Implementați o funcție recursivă *id3* care construiește arbori de decizie pe baza unui **set de date X** și a unei **mulțimi de atribute A**.
    
  Trebuie avute în vedere următoarele aspecte:
  * dacă toate exemplele din X aparțin unei singure clase C, atunci se construiește un nod frunză etichetat cu acea clasă C
  * dacă nu mai exista atribute, atunci construiește nodul frunză etichetat cu cea mai frecventă clasă din X
    
  În caz contrar:
  * se alege atributul $a^*$ care aduce cel mai mai mare câștig informațional și se construiește un *nod intermediar* corespunzător acestuia.

  $$
    entropy(X) = -\sum_{c \in C}\frac{|X_c|}{|X|}log_2\frac{|X_c|}{|X|}
  $$
  $$
    gain(X, a) = entropy(X) - \sum_{v_{j} \in vals(a)} \frac{|X_{i/j}|}{|X|}entropy(X_{i/j})
  $$
  $$
    a^* = \underset{a \in A}{\operatorname{arg max}}\ gain(X, a)
  $$

  * pentru fiecare valoare posibilă $v_j$ a lui $a^*$ se construiește un subarbore apelând recursiv funcția *id3* pentru:

$$
  X_j = \{x \in X|a^*(x) = v_j\}\\
  A_{new} = A\setminus\{a^*\}
$$

În cazul prezentat mai sus, entropia este utilizată pentru a măsura randomness-ul din date. Intuitiv, cu cât un eveniment are probabilitate mai mare să se întâmple atunci acesta va avea o entropia din ce în ce mai mică. Prin modul în care se construiește arborele *ID3* se încearcă reducerea entropiei alegând la fiecare pas atributele care ne ofera cea mai multă informație. Cât considerați că este entropia într-un *nod frunză*?

In [217]:
def countElems(X,elem):
  count = 0
  vals = list(x["CLASS"] for x in X)
  for i in vals:
    if i == elem:
      count = count + 1
  
  return count

def findSubset(X,Att,Value):
  subset = []
  for x in X:
    if x[Att] == Value:
      subset.append(x) 
  return subset

def findValuesfromAtt(X,Att):
    values = []
    for x in X:
      values.append(x[Att])
    return set(values)

In [287]:
classes, attributes, examples = getDataSet(DATASET_NAME)

def mostFrequentClass(X):
    # TODO Cerință 2
    return most_frequent(list(x["CLASS"] for x in X))

def entropy(X):
    # TODO Cerință 2
    if countElems(X,'yes') == 0 and not countElems(X,'no') == 0:
      return - countElems(X,'no') / len(X) * log2(countElems(X,'no') / len(X))
    elif not countElems(X,'yes') == 0 and countElems(X,'no') == 0:
      return - countElems(X,'yes') / len(X) * log2(countElems(X,'yes') / len(X))
    elif countElems(X,'yes') == 0 and countElems(X,'no') == 0:
      return 0
    else:
      return - (countElems(X,'yes') / len(X) * log2(countElems(X,'yes') / len(X)) + countElems(X,'no') / len(X) * log2(countElems(X,'no') / len(X)))

def gain(X, A):
    # TODO Cerință 2
    att_dict = {}
    inf_gain_dict = {}

    for a in A:
      att_dict[a] = findValuesfromAtt(examples,a)

    for a in A:
      H_att = 0
      for v in att_dict[a]:
        subset = findSubset(examples,a,v)
        H_val = entropy(subset)
        H_att = H_att + len(subset) / len(examples) * H_val
      inf_gain_dict[a] = H_att

    # print(inf_gain_dict)
    max_H = max(list(inf_gain_dict.values()))
    max_H_pos = list(inf_gain_dict.values()).index(max_H)

    return list(inf_gain_dict.keys())[max_H_pos]

def id3(X, A, m ='fail'):
    if len(A) == 0:
      n = Node(m)
      return n
    
    attribute = gain(X,A)
    n = Node(attribute)
    A.remove(attribute)

    m = mostFrequentClass(X)

    for v in findValuesfromAtt(X,attribute):
      subset_v = findSubset(X,attribute,v)
      n.children[v] = id3(subset_v,deepcopy(A),m)

    return n

def evaluate(tree, example):
    # TODO Cerință 2
    # Functia intoarce clasa prezisa de arborele `tree` pentru exemplul `example`
    curr_node = tree
    ex_attributes = list(example.keys())
    ex_attributes.remove('CLASS')

    while not len(curr_node.children) == 0:
      curr_node = curr_node.children[example[curr_node.label]]
    
    return curr_node.label

def precision(tree, X, c):
    prec = 0
    predicted_ct = 0
    
    for ex in X:
        pred_c = evaluate(tree, ex)
        if pred_c == c:
            predicted_ct += 1
            if ex['CLASS'] ==c:
                prec += 1
    
    if predicted_ct != 0:
        return prec / predicted_ct
    return 0

def recall(tree, X, c):
    X_c = list(filter(lambda ex: ex['CLASS'] == c, X))
    recall = 0
    
    for ex in X_c:
        pred_c = evaluate(tree, ex)
        if pred_c == c:
            recall += 1
            
    recall /= len(X_c)
    return recall

def accuracy(tree, X):
    count = 0
    for x in X:
      if evaluate(tree, x) == x['CLASS']:
        count += 1
        
    return 1.0 * count / len(X)


classes, attributes, examples = getDataSet(DATASET_NAME)
tree = randomTree(4, examples, attributes)
print('=== RANDOM TREE ===')
print('Precision:' ,precision(tree, examples, 'yes'))
print('Recall: ', recall(tree, examples, 'yes'))
print('Accuracy: ' , accuracy(tree, examples))

print()
classes, attributes, examples = getDataSet(DATASET_NAME)
tree = id3(examples,attributes)
# print(tree.display(""))
print('=== ID3 ===')
print('Precision:' ,precision(tree, examples, 'yes'))
print('Recall: ', recall(tree, examples, 'yes'))
print('Accuracy: ' , accuracy(tree, examples)) 

=== RANDOM TREE ===
Precision: 0.6666666666666666
Recall:  0.6666666666666666
Accuracy:  0.5714285714285714

=== ID3 ===
Precision: 0.7777777777777778
Recall:  0.7777777777777778
Accuracy:  0.7142857142857143


3. [4 pct] Implementați clasificator de tip pădure de arbori aleatori construind *n* arbori de adâncime maximă *d* fiecare dintre aceștia pornind de la o submulțime aleatoare a lui X.

    Folosiți funcția *randomTree* de la cerința 1.
  * Porniți de la *n = 100*, *d = 3* și submulțimi formate din 50% din elementele lui X alese la întamplare și experimentați cu acești hiperparametrii.
  * Pentru predicția clasei pentru obiecte noi alegeți clasa majoritară
  * Comparați rezultatele obținute folosind un singur arbore construit cu ID3 și o pădure de arbori aleatori. Discuție după *zgomot*, *overfitting*.

**Nu stiu ce sa fac in cazul in care nu gaseste o valoare pentru atribut. In sampling nu e garantat ca toate valorie se regasesc in tree. :(**

In [285]:
from statistics import mean 

classes, attributes, examples = getDataSet(DATASET_NAME)

def randomForest(X, n, d):
    forest = []
    
    for i in range(n):
        sample = random.sample(X, int(len(X) / 2))

        A = list(sample[0].keys())
        A.remove("CLASS")
        forest.append(randomTree(d, sample, A))
    
    return forest
    
frst = randomForest(examples, 100, 4)

results = []
med_prec = []
med_recall = []
med_accuracy = []

for i in range(len(frst)):
  results.append(evaluate(frst[i], examples[3]))
  med_prec.append(precision(frst[i], examples, 'yes'))
  med_recall.append(recall(frst[i], examples, 'yes'))
  med_accuracy.append(accuracy(frst[i], examples))

print('Result: ', most_frequent(results))
print('Med Precision: ', mean(med_prec))
print('Med Recall: ', mean(med_recall))
print('Med Accuracy: ', mean(med_accuracy))

KeyError: ignored

## 7. Set de date

În cadrul acestui laborator se vor folosi seturile de date [Car Evaluation](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation), [Chess](https://archive.ics.uci.edu/ml/datasets/Chess+%28King-Rook+vs.+King-Pawn%29) și [Tennis](https://www.kaggle.com/fredericobreno/play-tennis).

Aceste seturi de date sunt "ușor" modificate astfel încât pe prima linie să se afle atributele și labelul/clasa din care face parte fiecare exemplu.

Atributele datasetului *Chess* nu sunt intuitive, iar dacă doriți să aflați mai multe informații despre acestea, puteți accesa link-ul de [aici](https://pdfs.semanticscholar.org/db58/88d3f373aff2c6bd7b2f956b81c6896874a9.pdf?_ga=2.193733611.798337455.1582711694-486327444.1582711694).

## 8. Extra

### 8.1 ID3 exemplu
Un exemplu mai detaliat pentru construcția arborelui de decizie ID3 se poate găsi [aici](https://github.com/cs-pub-ro/ML/blob/master/lab/lab2/id3_example.pdf).

### 8.2 CART
Un alt algoritm utilizat poartă denumirea de CART (eng. Classification and Regression Tree). Dacă **ID3** utilizeaza **câștigul informațional (eng. information gain)**, **CART** utilizeaza o altă metrică numită **index-ul Gini (eng. Gini index sau Gini impurity)**.

Pentru implementare, se urmăresc exact aceeași [pași ca la ID3](#scrollTo=rjYqUPSbe1gG), singura diferentă fiind modul în care se calculează atributul utilizat într-un *nod intermediar*.
$$
Gini(X, a) = 1 - \sum_{c \in C}{p(c | attr(X) = a) ^2}
\\
a^* = \underset{a \in A}{\operatorname{arg min}}\ Gini(X, a)
$$